In [ ]:
import torch
weights = torch.load('extracted_weights.pth')

/home/mohamed/CFU-Playground/env/conda/envs/bibench/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
encoded_vectors = []
for i, block_data in enumerate(weights):
    if 'encoded_vector' in block_data:
        encoded_vector = block_data['encoded_vector']
        # Convert to NumPy array for easier handling
        encoded_vector_numpy = encoded_vector.numpy() #.tolist() #.astype(np.float32)
        encoded_vectors.append(encoded_vector_numpy)

# encoded_vectors is now a list where each element is a NumPy array
# representing the encoded_vector for that block.

# Double-check the structure:
print(f"Number of blocks with encoded_vectors: {len(encoded_vectors)}")
print(f"Shape of the first encoded_vector: {encoded_vectors[0].shape}")  # Should be (256, 16)

Number of blocks with encoded_vectors: 24
Shape of the first encoded_vector: (5120,)


In [ ]:
print(encoded_vectors)

[array([184, 194,  46, ...,  85, 126, 215], dtype=uint8), array([ 51,   0,  18, ...,  59,   6, 110], dtype=uint8), array([10, 33, 35, ..., 25, 98, 41], dtype=uint8), array([ 17,  58, 180, ...,  87,  71, 118], dtype=uint8), array([150, 188,  22, ...,  44,  59,  40], dtype=uint8), array([243,  77,  81, ...,   5,  30,  18], dtype=uint8), array([243, 166,  66, ...,  42, 142,  40], dtype=uint8), array([41, 37, 28, ..., 12, 43, 49], dtype=uint8), array([  3,  28,  33, ..., 220,  16, 192], dtype=uint8), array([ 96, 132,  17, ...,  68,   7,  59], dtype=uint8), array([240,  53, 199, ...,  55, 229,  39], dtype=uint8), array([210,  11,  30, ...,   3,  59,  55], dtype=uint8), array([ 52,  45, 184, ...,  38, 167,   1], dtype=uint8), array([ 38,  39, 194, ..., 158, 132, 135], dtype=uint8), array([ 47, 211, 212, ..., 165, 113, 232], dtype=uint8), array([ 21,  51, 142, ..., 144,  12, 106], dtype=uint8), array([ 67,  88, 101, ...,  28,  64,  47], dtype=uint8), array([100,  15,  54, ...,  32,  81, 144],

In [ ]:
import numpy as np


# Function to pack 4 int8 values into 1 int32
def pack_int8_to_int32(arr):
    # Ensure the array length is a multiple of 4
    if len(arr) % 4 != 0:
        raise ValueError("Array length must be a multiple of 4")
    
    # Reshape the array to group every 4 int8 values
    reshaped = arr.reshape(-1, 4)
    
    # Shift each int8 value to its correct position and combine
    packed = (
                (reshaped[:, 0].astype(np.int32) << 24) |  # Shift first byte to MSB
                (reshaped[:, 1].astype(np.int32) << 16) |   # Shift second byte
                (reshaped[:, 2].astype(np.int32) << 8) |   # Shift third byte
                (reshaped[:, 3].astype(np.int32)) )         # No shift for the fourth byte
    return packed

# Pack each array in the list
packed_arrays = [pack_int8_to_int32(arr) for arr in encoded_vectors]



Original: tensor(87, dtype=torch.uint8)
Packed: [-1195233671   624451639   342284047 ...  2010321878   123210024
 -1504346409]
Original: tensor(145, dtype=torch.uint8)
Packed: [  855642827  1689392676  1595189274   229117446  -548278746  -366473464
 -1947734236  -372435265   352687367 -1056836858   185339671   588455181
  1463729960 -1354242781 -1054445683  -248392315  1880785272   638786854
   608409620  1420582419   527080451   528626523  1873008673  2084652863
 -1240328425     2627369   538079791   623195109    16327189 -1251111419
   152443681   367206466   940229124   389029136  2082479887   577801991
   561204231   405163014   487335513  1947027826   287234098 -2058665801
 -1676653041   413183009   641408341  -234021281  1212948495   445777434
  -383556109  2017075988 -1960634593  2035668276  2114810127 -1693112204
   433058313   402717981   342484649   114920113   303201551   438912951
   286597260   370019374   388281121  1846953995    69128901  1223304800
  -618061549   304401

In [16]:
print(encoded_vectors[5][0:8])
print(packed_arrays[5][0:4])

[243  77  81 207  47  59  45  41]
[ -213036593   792407337 -1205029862  1378164368]


In [21]:
import numpy as np
from tensorflow.lite.tools import flatbuffer_utils
from tensorflow.lite.python import schema_py_generated as schema_fb

def load(path):
    return flatbuffer_utils.read_model_with_mutable_tensors(path)

def save(model , path):
    flatbuffer_utils.write_model(model, path)


def modify_conv2d_to_custom(model):
    # Create a new custom op code
    custom_op_code = schema_fb.OperatorCodeT()
    custom_op_code.customCode = "CONV_XOR"
    custom_op_code.builtinCode = schema_fb.BuiltinOperator.CUSTOM
    model.operatorCodes.append(custom_op_code)
    custom_op_index = len(model.operatorCodes) - 1

    for subgraph in model.subgraphs:
        for i ,operator in enumerate(subgraph.operators):
            op_code = model.operatorCodes[operator.opcodeIndex]
            if op_code.builtinCode == schema_fb.BuiltinOperator.CONV_2D and i>15:
                # Modify the operator to use your custom op
                operator.opcodeIndex = custom_op_index
                
                # Convert builtin options to custom options
                #to do

    return model






def modify_conv_weights(model,packed_arrays):
    b=1
    i=1
    for subgraph in model.subgraphs:
        for operator in subgraph.operators:
            i+=1
            op_code = model.operatorCodes[operator.opcodeIndex]
            #if op_code.customCode == "CONV_XOR":
            if op_code.builtinCode == schema_fb.BuiltinOperator.CONV_2D and i>15:
    
                # Assuming weights are the second input to the conv operator
                weight_tensor_index = operator.inputs[1]
                weight_tensor = subgraph.tensors[weight_tensor_index]
                new_name = f'conv_xor_w_{b}'
                weight_tensor.name=new_name
               # print(new_name.encode('utf-8'))
                buffer = model.buffers[weight_tensor.buffer]
                # weights = np.frombuffer(buffer.data, dtype=np.float32)
                # weights = weights.reshape(weight_tensor.shape)
               # print(type(weight_tensor.shape))
                weights=packed_arrays[b-1]
                b+=1
                #print(type(np.array(weights.shape)))
                #weight_tensor.shape = np.array(weights.shape)

                # # Modify data type
                #weight_tensor.type = schema_fb.TensorType.INT32
                buffer.data = weights.tobytes()

    return model

    
def process_model(path,packed_arrays):
    model=load(path)
    #model=modify_conv2d_to_custom(model)
    model=modify_conv_weights(model,packed_arrays)
    save(model,"dfsmn_cbnn.tflite")
  
        



In [23]:
process_model('model_8_16_float32.tflite',packed_arrays)